# Protein sequence playing around
### Continued

In [1]:
import numpy as np
import requests
import re
import xml.etree.ElementTree as ET

import matplotlib.pyplot as plt

### Load data
Sequences were previously taken from the RCSB website

In [2]:
import pickle
with open ('fastas', 'rb') as read:
    all_seq = pickle.load(read)

Full vocabulary

In [4]:
unique = set()
for s in all_seq:
    for a in s:
        unique.update(a)
''.join(unique)

'IWPulCREDMQSNGVFYHLnKTXAU'

In [39]:
# conversion maps
n_to_aa = {i:v for i,v in enumerate(unique)}
aa_to_n = {v:i for i,v in enumerate(unique)}

Truncating short sequences

In [5]:
X_raw = [x for x in all_seq if len(x)>50]
len(X_raw)

29778

### Model

In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.preprocessing import sequence

Using TensorFlow backend.


In [7]:
# vocabulary (number of one-hot-encoded columns)
voc = len(unique)
# window size ('memory')
win = 25

model = Sequential()
model.add(LSTM((64),input_shape=(win, voc),return_sequences=True))
model.add(LSTM((128),input_shape=(win, voc),return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(256,activation='relu'))   
model.add(Dense(voc,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 25, 64)            23040     
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               98816     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 25)                6425      
Total params: 161,305
Trainable params: 161,305
Non-trainable params: 0
_______________

In [8]:
# prepare X and y. X_total is a tuple of 'x' and 'y' 
# everything is essentially taken from the same sequences 'X_raw'
X_total = []
for x in X_raw:
    X_total.extend([(x[i:i+win],x[i+win]) for i in range(len(x)-(win))])
len(X_total)

7648920

In [34]:
# input and labels generator
def one_hot_generator(X_all,batch_size,window,vocabulary,iterations=10):
    # run indefinitely
    while (True):
        # generate new indices
        indices = np.random.choice(range(len(X_all)),size=batch_size*iterations,replace=False)
        for i in range(iterations):
            # subset of indices, generate one-hot-encoded X and y
            total = np.array(X_all)[indices[i*batch_size:(i+1)*batch_size]]

            X = np.zeros((batch_size,window,vocabulary),dtype=int)
            y = np.zeros((batch_size,vocabulary),dtype=int)

            for i in range(batch_size):
                xy = total[i]
                X[i][range(win),[aa_to_n[i] for i in xy[0]]] = 1
                y[i][aa_to_n[xy[1]]] = 1  

            yield X,y

### Train model

In [44]:
model.fit_generator(one_hot_generator(X_total,25000,win,voc),steps_per_epoch=10,epochs=100)

Epoch 1/100
10/10 [==============================] - 191s 19s/step - loss: 2.8081 - acc: 0.1061
Epoch 2/100
10/10 [==============================] - 188s 19s/step - loss: 2.8073 - acc: 0.1072
Epoch 3/100
10/10 [==============================] - 184s 18s/step - loss: 2.8089 - acc: 0.1061
Epoch 4/100
10/10 [==============================] - 178s 18s/step - loss: 2.8078 - acc: 0.1072
Epoch 5/100
10/10 [==============================] - 177s 18s/step - loss: 2.8079 - acc: 0.1067
Epoch 6/100
10/10 [==============================] - 180s 18s/step - loss: 2.8047 - acc: 0.1073
Epoch 7/100
10/10 [==============================] - 184s 18s/step - loss: 2.8063 - acc: 0.1074
Epoch 8/100
10/10 [==============================] - 181s 18s/step - loss: 2.8057 - acc: 0.1076
Epoch 9/100
10/10 [==============================] - 183s 18s/step - loss: 2.8056 - acc: 0.1080
Epoch 10/100
10/10 [==============================] - 183s 18s/step - loss: 2.8058 - acc: 0.1072
Epoch 11/100
10/10 [===================

10/10 [==============================] - 178s 18s/step - loss: 2.7860 - acc: 0.1152
Epoch 86/100
10/10 [==============================] - 183s 18s/step - loss: 2.7869 - acc: 0.1148
Epoch 87/100
10/10 [==============================] - 183s 18s/step - loss: 2.7880 - acc: 0.1138
Epoch 88/100
10/10 [==============================] - 185s 18s/step - loss: 2.7872 - acc: 0.1144
Epoch 89/100
10/10 [==============================] - 179s 18s/step - loss: 2.7888 - acc: 0.1133
Epoch 90/100
10/10 [==============================] - 176s 18s/step - loss: 2.7859 - acc: 0.1150
Epoch 91/100
10/10 [==============================] - 177s 18s/step - loss: 2.7864 - acc: 0.1155
Epoch 92/100
10/10 [==============================] - 185s 18s/step - loss: 2.7881 - acc: 0.1145
Epoch 93/100
10/10 [==============================] - 181s 18s/step - loss: 2.7854 - acc: 0.1149
Epoch 94/100
10/10 [==============================] - 179s 18s/step - loss: 2.7863 - acc: 0.1146
Epoch 95/100
10/10 [=======================

### Evaluate model

In [45]:
model.evaluate_generator(one_hot_generator(X_total,10000,win,voc),steps=10)

[2.780729079246521, 0.11845999956130981]